In [ ]:
'''!pip install --upgrade pip setuptools wheel
!git clone https://github.com/mikel-brostrom/yolo_tracking.git  # clone repo
%cd yolo_tracking
!pip install -e .'''

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
import os
os.getcwd()

In [ ]:
# get the test video from the repo
!wget -nc https://github.com/mikel-brostrom/yolo_tracking/releases/download/v.2.0/test.avi
# extract 3 seconds worth of video frames of it
!yes | ffmpeg -ss 00:00:00 -i test.avi -t 00:00:01 -vf fps=30 out.avi

In [ ]:
!python tracking/track.py --yolo-model yolov8n.pt --reid-model osnet_x0_25_msmt17.pt --source out.avi --save --conf 0.2

In [ ]:
# Compress the video file to a format supported by Google Colab (mpeg4 (native) -> h264 (libx264))
!ffmpeg -i runs/track/exp/out.avi -vf fps=30 -vcodec libx264 output.mp4

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

In [ ]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# 원 tutorial에 있던 명령어(실행X)
!python tracking/val.py --tracking-method deepocsort --benchmark MOT17-mini --imgsz 320 --conf 0.2

# Readme 기반으로 재작성한 실행 명령어

In [ ]:
# Track YOLO model
!python tracking/track.py --yolo-model yolov8n.pt --tracking-method deepocsort --reid-model osnet_x0_25_msmt17.pt --source out.avi 

In [ ]:
# saves dets and embs under ./runs/dets_n_embs separately for each selected yolo and reid model
!python tracking/generate_dets_n_embs.py --source ./assets/MOT17-mini/train --yolo-model yolov8n.pt yolov8s.pt --reid-model tracking/weights/osnet_x0_25_msmt17.pt

In [ ]:
# generate MOT challenge format results based on pregenerated detections and embeddings for a specific trackign method
!python tracking/generate_mot_results.py --dets yolov8n --embs osnet_x0_25_msmt17 --tracking-method botsort

In [ ]:
# uses TrackEval to generate MOT metrics for the tracking results under ./runs/mot/<dets+embs+tracking-method>
!python tracking/val.py --benchmark MOT17-mini --dets yolov8n --embs osnet_x0_25_msmt17 --tracking-method botsort